## 1. Install / import required package

In [ ]:


import datetime

from tweetkit.auth import BearerTokenAuth
from tweetkit.client import TwitterClient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.2 MB/s 


## 2. Set authentication
You can find the acount information [here](https://docs.google.com/document/d/19l7dvUETIGpNuhvUZOxwqRXz0KXcPpPF/edit?usp=share_link&ouid=101933136194359097471&rtpof=true&sd=true).
We only need to input the **bearer token**. 

In [ ]:

bearer_token  = '.........'
auth = BearerTokenAuth()
auth.bearer_token = bearer_token
client = TwitterClient(auth=auth)

## 3. Set query parameters


1. Learn how to build a query [here](https://developer.twitter.com/en/docs/twitter-api/tweets/counts/integrate/build-a-query).

2.  You can find more details about the parameters [here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all).


In [ ]:

## ISO-8601 format
start_time_ = '2022-09-30T00:00:01Z' 
end_time_ = '2022-09-30T00:02:00Z'

# build a query e.g.'(#Ian OR "Hurricane Ian" OR #HurricaneIan) -is:retweet has:geo lang:en',
keywords = ['#foodpoisoning', '#stomachache', '"food poison"', '"food poisoning"', 'stomach', 'vomit', 'puke', 'diarrhea', '"the runs"']
search_keywords = ' OR '.join(keywords) # search_keywords: #foodpoisoning OR #stomachache OR "food poison" OR "food poisoning" OR stomach OR vomit OR puke OR diarrhea OR "the runs"

other_conditions = '-is:retweet lang:en' # has:geo

final_query = '('+search_keywords+') '+ other_conditions

# set other parameters
tweet_fields_ = ['attachments', 'author_id', 'context_annotations', 'conversation_id',
                 'created_at', 'edit_controls', 'edit_history_tweet_ids', 'entities', 'geo',
                 'id', 'in_reply_to_user_id', 'lang', 'possibly_sensitive', 
                 'public_metrics', 'referenced_tweets', 'reply_settings', 'source', 'text','withheld']

expansions_ =['attachments.media_keys', 'attachments.poll_ids', 'author_id',
              'edit_history_tweet_ids', 'entities.mentions.username', 'geo.place_id',
              'in_reply_to_user_id', 'referenced_tweets.id','referenced_tweets.id.author_id']

media_fields_= ['alt_text', 'media_key', 'preview_image_url','public_metrics', 'type', 'url']
poll_fields_ = ['duration_minutes', 'end_datetime', 'id', 'options', 'voting_status']
user_fields_ = ['created_at', 'description', 'entities', 'id', 'location', 'name',
                'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics',
                'url', 'username', 'verified', 'withheld']

place_fields_ = ['contained_within', 'country', 'country_code', 'full_name', 'geo', 'id','name', 'place_type']

"""See more details about the params in tweets_fullarchive_search here: https://tweetkit.readthedocs.io/en/latest/tweetkit.requests.html#tweetkit.requests.tweets.Tweets.tweets_fullarchive_search"""


paginator = client.tweets.tweets_fullarchive_search(
    query = final_query,
    start_time = start_time_,
    end_time = end_time_,
    max_results = 100,
    paginate = True,
    tweet_fields = tweet_fields_ ,
    expansions = expansions_ ,
    media_fields = media_fields_ ,
    poll_fields = poll_fields_ ,
    user_fields = user_fields_ ,
)

tweets = []

start_time = datetime.datetime.strptime(start_time_, '%Y-%m-%dT%H:%M:%SZ')
end_time = datetime.datetime.strptime(end_time_, '%Y-%m-%dT%H:%M:%SZ')
percentage_remaining = 0.0

total_period = (end_time - start_time).total_seconds()

for tweet in paginator.content:
    created_at = datetime.datetime.strptime(tweet['data']['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ')
    # print(tweet)
    tweets.append(tweet)
    remaining_period = (end_time - created_at).total_seconds()
    percentage_remaining = round(remaining_period * 100 / total_period, 2)
    print('\rTweet Count: {:3.0f}%, {}'.format(percentage_remaining, len(tweets)), end='')
if percentage_remaining != 100.00:
    print('\rTweet Count: {:3.0f}%, {}'.format(100.00, len(tweets)), end='')



Tweet Count: 100%, 27